歪打误撞，非自己决策的开仓——赚一笔就跑

In [16]:
import tushare as ts

ts.set_token('409115656486e3f61481103ec280339511aee50ee9026f9b008985ea')
pro = ts.pro_api()

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\tushare\util\upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


# 黄金和比特币趋势

## 比特币数据

In [31]:
ts.set_token("409115656486e3f61481103ec280339511aee50ee9026f9b008985ea")

pro = ts.pro_api()
df = pro.btc_pricevol(start_date='20180101', end_date='20180801')

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\tushare\util\upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [32]:
df.head()

,date,price,volume
0,20180801,7579.056837,5.759918e+08
1,20180731,7915.267761,7.305237e+08
2,20180730,8128.549310,6.716885e+08
3,20180729,8193.888653,2.656748e+08
4,20180728,8168.527643,2.808106e+08


## 黄金数据

# 期货-股票联动
农产品、贵金属

# 代码调试

In [1]:
def get_conzt_num(stock_code):
    df = get_hdata(stock_code)
    close_list = list(df["close"])
    pct_list = [round(close_list[i + 1] / close_list[i] * 100 - 100, 2) for i in range(len(close_list) - 1)]
    reversed_percent = pct_list[::-1]
    num = len([item for item in itertools.takewhile(lambda x: x >= 9.9, reversed_percent)])
    return num

In [71]:
time="20200304"
"%s-%s-%s" %(time[:4],time[4:6],time[6:])

'2020-03-04'

In [2]:
import pandas as pd
import numpy as np
import os

wd = "D:/python_stock/historical_data"
test_file = "%s/000001.csv" %(wd)

In [3]:
df = pd.read_csv(test_file,index_col=0)

In [6]:
rev_df = df.iloc[::-1]

二项操作：
1）连续>+9.9的个数
2）根据new_stock记录，新开板个股 - 开板高度

In [ ]:
df = df[~df.index.str.startswith("688")]                             ### 排除科创板数据

In [ ]:
class SubNewStockMongo():
    """
    近端次新：60日，远端次新：180日，自动更新
    统计开板高度，提醒今日开板
    """

    def __init__(self,timeout=300):
        self.client = MongoClient()
        self.db = self.client["stock"]
        self.collection = self.db["new_stock"]
        self.concept_collection = self.db["stock_concept"]

    def get_ave_pchange(self, df, type):
        stock_list= []
        for item in self.collection.find({"type":type,"kb":True}):    #排除未开板新股
            stock_list.append(item["stock_code"])
        filter = df.loc[stock_list,:]
        os.chdir(os.path.join("D:/python_stock/涨停概念分析",judge_date()))
        filter.to_csv("%s次新.csv" %(type), encoding="utf_8_sig")
        ave_change = round(filter["changepercent"].sum()/len(stock_list), 2)
        print("开板%s次新的今日涨幅: %s" %(type,ave_change))
        return ave_change

    def find_one(self,stock_code):
        return self.collection.find_one({"stock_code":stock_code})

    def delete_one(self,stock_code):
        self.collection.delete_one({"stock_code":stock_code})

    def degrade(self,stock_code):
        self.collection.update({"stock_code":stock_code},{"$set":{"type":"远端"}},upsert=True)

    def update(self, stock_code, stock_name, ttm, issue_price):
        """
        ttm：开板时间（为什么当时要取这个名字）
        """
        if not self.find_one(stock_code):
            print("出现新股：%s\t%s，于%s上市" % (stock_code, stock_name, ttm))
            self.collection.update({"stock_code": stock_code},
                                   {"$set": {"stock_name": stock_name, "ttm": ttm, "issue_price": issue_price,"kb": False,"type":"近端"}},
                                    upsert=True)
        if self.collection.find_one({"stock_code": stock_code, "kb": False}):
            kbgd = get_kbgd(stock_code)
            if kbgd == 0:
                self.collection.update({"stock_code": stock_code},
                                       {"$set": {"stock_name": stock_name, "ttm": ttm, "issue_price": issue_price,"kb": False, "type":"近端"}},
                                        upsert=True)
            else:
                self.collection.update({"stock_code": stock_code},
                                       {"$set": {"stock_name": stock_name, "ttm": ttm, "issue_price": issue_price,"kb": True, "kbgd": kbgd, "type":"近端"}},
                                        upsert=True)
                self.concept_collection.update({"stock_code": stock_code},
                                               {"$addToSet": {"jqk_concept": "开板近端次新"}},
                                               upsert=True)
                print("%s %s今天开板，开板高度%s" % (stock_code, stock_name, kbgd))
        else:
            pass